In [1]:
import numpy as np
from pathlib import Path
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
import torchvision
from torchvision import *
from torch.utils.data import Dataset, DataLoader
from torchvision.io import read_image
from torchvision import datasets, models, transforms
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [3]:
import time
from tqdm import tqdm
import math

In [4]:
DATA_DIR = Path('./dataset/')

In [5]:
def limit_frequencies( fft, inner_limit=0, outer_limit=140 ):
    center = (fft.shape[1] / 2, fft.shape[0] / 2)
    for y in range(fft.shape[0]):
        for x in range(fft.shape[1]):
            r = math.sqrt( abs(center[0] - x) ** 2 + abs(center[1] - y) ** 2 )
            if r > outer_limit:
                fft[y,x] = 0
    
            if r < inner_limit:
                fft[y,x] = 0

    return fft

In [6]:
def calculate_2dft( input_img ):
    input = np.float32( input_img )[:,:,0]
    dft = np.fft.fft2(input)
    dft_shift = np.fft.fftshift(dft)
    ft = np.real(dft_shift)
    ft = limit_frequencies( ft, inner_limit=25, outer_limit=120)
    ft[int(ft.shape[0] / 2),int(ft.shape[1] / 2)] = 0
    
    return torch.Tensor(ft).reshape((1,ft.shape[0],ft.shape[1]))

# Datenset laden

In [7]:
train_dir = "./dataset/train/"
test_dir = "./dataset/test/"
train_classa_dir = "./dataset/train/moire/"
train_classb_dir = "./dataset/train/no_moire/"
test_classa_dir = "./dataset/test/moire/"
test_classb_dir = "./dataset/test/no_moire/"

In [8]:
# Create transform function
transforms_train = transforms.Compose([
    transforms.Lambda(calculate_2dft)
])
transforms_test = transforms.Compose([
    transforms.Lambda(calculate_2dft)
])

In [9]:
train_dataset = datasets.ImageFolder(train_dir, transforms_train)
test_dataset = datasets.ImageFolder(test_dir, transforms_test)

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=48, shuffle=True, num_workers=0)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=48, shuffle=False, num_workers=0)

# Modell trainieren

In [10]:
#device = torch.device('mps')
device = torch.device('cpu')

In [11]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()

        self.conv1 = nn.Conv2d(1, 6, 5, stride=2)
        self.conv2 = nn.Conv2d(6, 12, 5, stride=2)
        self.conv3 = nn.Conv2d(12, 35, 5, stride=2)

        self.flatten = nn.Flatten()
        
        self.fc1 = nn.Linear(35840, 20000)  # 5*5 from image dimension
        self.fc2 = nn.Linear(20000, 1000)
        self.fc3 = nn.Linear(1000, 2)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = self.flatten(x)

        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.sigmoid(self.fc3(x))
        return x

In [12]:
model = Net().to(device)

In [13]:
# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)

In [14]:
torch.backends.mps.is_available()

True

In [15]:
0 / 0

ZeroDivisionError: division by zero

In [16]:
#### Train model
train_loss=[]
train_accuary=[]
test_loss=[]
test_accuary=[]

num_epochs = 10   #(set no of epochs)
start_time = time.time() #(for showing time)
# Start loop
for epoch in range(num_epochs): #(loop for every epoch)
    print("Epoch {} running".format(epoch + 1)) #(printing message)
    """ Training Phase """
    model.train()    #(training model)
    running_loss = 0.   #(set loss 0)
    running_corrects = 0 
    # load a batch data of images
    with tqdm(total=len(train_dataloader)) as pbar:       
        for i, (inputs, labels) in enumerate(train_dataloader):
            inputs = inputs.to(device)
            labels = labels.to(device) 
            # forward inputs and get output
            optimizer.zero_grad()
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)
            # get loss value and update the network weights
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            running_corrects += torch.sum(preds == labels.data).item()

            pbar.update(1)
    
    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = running_corrects / len(train_dataset) * 100.
    # Append result
    train_loss.append(epoch_loss)
    train_accuary.append(epoch_acc)
    # Print progress
    print('[Train #{}] Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.format(epoch+1, epoch_loss, epoch_acc, time.time() -start_time))
    """ Testing Phase """
    model.eval()
    with torch.no_grad():
        running_loss = 0.
        running_corrects = 0
        for inputs, labels in test_dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            running_corrects += torch.sum(preds == labels.data).item()
        epoch_loss = running_loss / len(test_dataset)
        epoch_acc = running_corrects / len(test_dataset) * 100.
        # Append result
        test_loss.append(epoch_loss)
        test_accuary.append(epoch_acc)
        # Print progress
        print('[Test #{}] Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.format(epoch+1, epoch_loss, epoch_acc, time.time()- start_time))

Epoch 1 running


100%|█████████████████████████████████████████| 106/106 [10:28<00:00,  5.93s/it]


[Train #1] Loss: 0.0146 Acc: 48.7032% Time: 628.3222s
[Test #1] Loss: 0.0147 Acc: 49.1379% Time: 672.9256s
Epoch 2 running


100%|█████████████████████████████████████████| 106/106 [11:59<00:00,  6.79s/it]


[Train #2] Loss: 0.0145 Acc: 48.3073% Time: 1392.5499s
[Test #2] Loss: 0.0147 Acc: 49.0596% Time: 1430.5423s
Epoch 3 running


...

[Train #8] Loss: 0.0145 Acc: 48.3667% Time: 9300.4285s
[Test #8] Loss: 0.0147 Acc: 49.1379% Time: 9337.9717s
Epoch 9 running


100%|█████████████████████████████████████████| 106/106 [11:21<00:00,  6.43s/it]


[Train #9] Loss: 0.0145 Acc: 48.3865% Time: 10019.5405s
[Test #9] Loss: 0.0147 Acc: 49.2163% Time: 10057.1064s
Epoch 10 running


100%|█████████████████████████████████████████| 106/106 [12:03<00:00,  6.82s/it]


[Train #10] Loss: 0.0145 Acc: 48.4063% Time: 10780.4517s
[Test #10] Loss: 0.0147 Acc: 49.2163% Time: 10818.0385s
